In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Beachte Konvention: alles in Winkelsekunden rechnen

### Observed Zenith angle

In [2]:
time_top = 15 * (np.array([11*60*60+42*60+47, 11*60*60+44*60+8, 11*60*60+56*60+50,
                     11*60*60+57*60+37, 12*60*60+4*60+53, 
                     12*60*60+6*60 +3]) - 2*3600) # in arcsec, UT
z_b_top = np.array([39*3600+9*60+32, 39*3600+3*60+16, 38*3600+6*60+6, 38*3600+3*60+7, 37*3600+35*60+57, 37*3600+32*60+2])

time_bottom = 15 * (np.array([11*60*60+49*60+8, 11*60*60+51*60+3,11*60*60+59*60+29,
                     12*60*60+1*60+11]) - 2*3600 )# in arcsec, UT
z_b_bottom = np.array([39*3600+12*60+26, 39*3600+4*60+52, 38*3600+29*60+1, 38*3600+21*60+33])

time_bottom/(3600*15) +2

array([11.81888889, 11.85083333, 11.99138889, 12.01972222])

### Almanac

In [3]:
GMST_0UT = (14*3600 + 52*60 + 28.7584) * 15 # in arcsecs
alpha_5 = 2.8197997 * 3600 * 15
alpha_6 = 2.8841898 * 3600 * 15
delta_5 = 16.262748 * 3600
delta_6 = 16.545533 * 3600
phi_1 = 15*60 + 51.58
phi_2 = 8.72 * 3600

l = 13 * 3600 + 17*60 + 42.5 # in arcsecs

### Corrections

#### Index error

In [4]:
z_0 = np.array([85+35/60+37/3600, 85+35/60+59/3600, 85+35/60+26/3600, 85+35/60+28/3600]) # in deg
z_180 = np.array([274+23/60+44/3600, 274+23/60+53/3600, 274+23/60+24/3600, 274+22/60+41/3600]) # in deg
i = np.mean((360 - (z_0 + z_180))/2)

#### Refraction

In [5]:
# z_b is in '' but we want °
R_mean_top = 1/np.tan(((90-z_b_top/3600) + (7.31)/ (90-z_b_top/3600+4.4)) * np.pi/180)
R_mean_bottom = 1/np.tan(((90-z_b_bottom/3600) + (7.31)/ (90-z_b_bottom/3600+4.4)) * np.pi/180)

P_mean = 0.5*(101.83 + 101.80 ) # kPa
T_mean = 0.5*(11 + 13) #°C

R_top = R_mean_top *(P_mean/101 * (273+T_mean)/283)
R_bottom = R_mean_bottom *(P_mean/101 * (273+T_mean)/283)

#### Transform to center of solar disk

In [6]:
z_top = z_b_top + i + R_top - phi_1 - phi_2
z_bottom = z_b_bottom + i + R_bottom + phi_1 - phi_2

In [7]:
z = np.concatenate((z_top, z_bottom))
time = np.concatenate((time_top, time_bottom))

### Almanac

### Interpolation alpha

In [8]:
alpha = alpha_5/(3600*15) + time/(24*3600*15) * (alpha_6/(3600*15) - alpha_5/(3600*15))

print(alpha_5, alpha_6)

152269.18379999997 155746.24920000002


### Interpolation delta

In [9]:
delta = delta_5/(3600) + time/(24*3600*15) * (delta_6/(3600) - delta_5/(3600))

### Hour Angle in arcsec

In [10]:
tau = GMST_0UT + time * (366.24/365.24) - l - alpha

In [11]:
Y = np.arctan(np.cos(tau*np.pi/648000) / np.tan(delta * np.pi/648000)) # rad
X = np.sin(delta*np.pi/648000)*np.sqrt(1 + (np.cos(tau*np.pi/648000)/ np.tan(delta*np.pi/648000))** 2)

In [12]:
b = (52 + 27/60 + 24/3600)*np.pi/180 # rad
print((Y + b) > np.pi/2)

[ True  True  True  True  True  True  True  True  True  True]


-> First solution for b!

In [13]:
b = (np.pi - np.arcsin(np.cos((z)*np.pi/648000)/X) - Y) * 180/np.pi

In [14]:
np.cos(2)

-0.4161468365471424